In [5]:
!git clone https://github.com/victorsimrbt/Chess-Deep-Learning.git
!pip install chess

fatal: destination path 'Chess-Deep-Learning' already exists and is not an empty directory.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:0000:01


In [10]:
import os
import chess
import numpy as np
import pandas as pd
import keras
import time

from keras import callbacks, optimizers
from keras.layers import (LSTM, BatchNormalization, Dense, Dropout, Flatten,
                          TimeDistributed)
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model, model_from_json
from IPython.display import clear_output
from matplotlib import pyplot as plt

flatten = lambda l: [item for sublist in l for item in sublist]

os.chdir('/kaggle/working/Chess-Deep-Learning')
df = pd.read_csv('chess_normalized.csv')
data = df['moves'].tolist()

lendata= len(data)
data = data[:100]

chess_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0],
}

def make_matrix(board): 
    pgn = board.epd()
    foo = []  
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []  
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo

def translate(matrix,chess_dict):
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(chess_dict[term])
        rows.append(terms)
    return rows

def data_setup():
    X = []
    y = []
    for game in range(len(data)):
        data[game] = data[game].split()
        board = chess.Board()
        for move in range(len(data[game])):
            legal_moves = str(board.legal_moves)[36:-2].replace(',','').split()
            matrix = make_matrix(board.copy())
            translated = translate(matrix,chess_dict)
            X.append(translated)
            try:
                move_index = legal_moves.index(data[game][move])
                y_value = move_index / len(legal_moves)
            except ValueError:
                print(f"Move '{data[game][move]}' not legal in game {game}, move {move}. Skipping...")
                continue  # Skip to the next move if the current move is not legal
            board.push_san(data[game][move])
            y.append(legal_moves.index(data[game][move])/len(legal_moves))
    return X,y

def initialize_network():
    model = Sequential()
    model.add(Conv2D(filters=10, kernel_size=1, activation='relu', input_shape=(8,8,12)))
    model.add(MaxPooling2D(pool_size=2, strides=None))
    model.add(Conv2D(filters=10, kernel_size=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=None))
    model.add(Conv2D(filters=10, kernel_size=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=None))
    model.add(Conv2D(filters=10, kernel_size=1, activation='relu'))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(1,activation = 'sigmoid'))
    return model

print('Preparing Data...')
start = time.time()
X,y = data_setup()
one_game_time = time.time()-start
X = np.array(X)
X_test = X[:int(len(X)*0.1)]
X_train = X[int(len(X)*0.1):]
y_test = y[:int(len(X)*0.1)]
y_train = y[int(len(X)*0.1):]
print('Initalizing Network...')
model = initialize_network()
print('Compiling Network...')

model.compile(optimizer='Nadam', loss='mse')
dirx = '//Users/elizabethtai/Desktop/Files/Programs/ML/Best Models Stocks'
os.chdir(dirx)
h5 = 'chess' + '_best_model' + '.h5'
checkpoint = callbacks.ModelCheckpoint(h5,
                                           monitor='val_loss',
                                           verbose=0,
                                           save_best_only=True,
                                           save_weights_only=True,
                                           mode='auto',
                                           period=1)
es = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5000/10)
callback = [checkpoint,es]
json = 'chess' + '_best_model' + '.json'
model_json = model.to_json()
with open(json, "w") as json_file:
    json_file.write(model_json)
print('Training Network...')
history = model.fit(X_train,y_train,epochs = 1000,verbose = 2,validation_data = (X_test,y_test),callbacks = callback)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

Preparing Data...
Move 'Ke7' not legal in game 2, move 31. Skipping...
Move 'Bc5+' not legal in game 2, move 32. Skipping...
Move 'Kf6' not legal in game 2, move 33. Skipping...
Move 'Bxf8' not legal in game 2, move 34. Skipping...
Move 'Kg6' not legal in game 2, move 35. Skipping...
Move 'Bxg7' not legal in game 2, move 36. Skipping...
Move 'Kxg7' not legal in game 2, move 37. Skipping...
Move 'dxe6' not legal in game 2, move 38. Skipping...
Move 'Kh6' not legal in game 2, move 39. Skipping...
Move 'exf7' not legal in game 2, move 40. Skipping...
Move 'Nf6' not legal in game 2, move 41. Skipping...
Move 'Rxh8' not legal in game 2, move 42. Skipping...
Move 'Nh5' not legal in game 2, move 43. Skipping...
Move 'Bxh5' not legal in game 2, move 44. Skipping...
Move 'Kg5' not legal in game 2, move 45. Skipping...
Move 'Rxh7' not legal in game 2, move 46. Skipping...
Move 'Kf5' not legal in game 2, move 47. Skipping...
Move 'Qf3+' not legal in game 2, move 48. Skipping...
Move 'Ke6' not leg

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


FileNotFoundError: [Errno 2] No such file or directory: '//Users/elizabethtai/Desktop/Files/Programs/ML/Best Models Stocks'

In [ ]:
chess_board = chess.Board()
for i in range(100):
    matrix = make_matrix(chess_board)
    board = translate(matrix,chess_dict)
    board = np.array(board)
    board = np.reshape(board,(1,8,8,12))
    pred = model.predict(board)
    legal_moves = str(chess_board.legal_moves)[36:-2].replace(',','').split()
    index = int(round((len(legal_moves)*pred)[0][0]))
    chess_board.push_san(legal_moves[index])
    if chess_board.is_checkmate():
        break
chess_board

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])

In [ ]:
data = df['moves'].tolist()
data.index(sorted(data,key=lambda point: len(point),reverse = True)[0])

In [ ]:
model.weights

In [ ]:
X[1]

In [ ]:
import chess
board = chess.Board()
def make_matrix(board): 
    pgn = board.epd()
    foo = [] 
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []  
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo

In [ ]:
import os
os.getcwd()
os.chdir('C:\\Users\\v_sim\\Desktop\\Files\\Data')

In [ ]:
import chess.pgn
pgn = open("Carlsen.pgn")
sides = []
games = []
length = 100
for i in range(length):
    try:
        if chess.pgn.read_game(pgn).mainline_moves():
            games.append(chess.pgn.read_game(pgn).mainline_moves())
            sides.append(chess.pgn.read_game(pgn).headers["White"])
    except:
        print(i,chess.pgn.read_game(pgn))
        pass
len(games)

In [ ]:
X = []
y = []
counter2 = 0
for game in games:
    board = chess.Board()
    white = sides[counter2]
    if white == 'Carlsen,Magnus':
        remainder = 0
    else:
        remainder = 1
    counter = 0
    for move in game:
        if counter % 2 == remainder:
            X.append(board.copy())
        board.push(move)
        if counter % 2 == remainder:
            y.append(board.copy())
        counter += 1
    counter2 += 1
X[0]

In [ ]:
chess_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1,0],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0,1],
}

def make_matrix(board): 
    pgn = board.epd()
    foo = []  
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []  
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo

def translate(matrix,chess_dict):
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(chess_dict[term])
        rows.append(terms)
    return rows

In [ ]:
import numpy as np
for i in range(len(X)):
    X[i] = translate(make_matrix(X[i]),chess_dict)
for i in range(len(y)):
    y[i] = translate(make_matrix(y[i]),chess_dict)
X = np.array(X)
y = np.array(y)

In [ ]:
import random
flatten = lambda l: [item for sublist in l for item in sublist]
def retranslate(action):
    board = []
    flatten_action = flatten(action)
    for i in range(len(flatten_action)):
        new_set = np.zeros((13,))
        max_index = list(flatten_action[i]).index(max(flatten_action[i]))
        new_set[max_index] = 1
        board.append(new_set)
    for i in range(len(board)):
        board[i] = new_chess_dict[tuple(board[i])]
    board = np.array(board).reshape(8,8)
    return board

def matrix2board(matrix):
    fen = []
    for row in matrix:
        foo2 = str()
        counter = 0
        counter2 = 0
        for thing in row:
            counter2+=1
            if thing == '.':
                counter+=1
            if counter2 == len(row) and thing == '.':
                foo2+=str(counter)
            if thing != '.':
                if counter > 0:
                    foo2+=str(counter)
                    counter = 0
                foo2+=thing
        fen.append(foo2)

    final_fentasy = str()
    for i in range(len(fen)):
        final_fentasy += fen[i]
        if i != len(fen)-1:
            final_fentasy += '/'
    return chess.Board(final_fentasy)

new_chess_dict = {}
for term in chess_dict:
    definition = tuple(chess_dict[term])
    new_chess_dict[definition] = term
    new_chess_dict[term] = definition

matrix = retranslate(random.choice(X))
board = matrix2board(matrix)
board